# Cleansing Meditations

In [ ]:
import os
import pandas as pd
import re
from datetime import timedelta

In [ ]:
expected_columns = ['teacher_id',
                    'meditation_id',
                    'med_list_batch_id',
                    'med_list_scrape_date',
                    'med_list_scrape_status',
                    'med_detail_batch_id',
                    'med_detail_scrape_date',
                    'med_detail_scrape_status',
                    'title',
                    'upload_date',
                    'duration',
                    'plays',
                    'rating',
                    'reviews',
                    'track_type',
                    'activity',
                    'suitable_for',
                    'topics',
                    'description',
                    'meditation_url',
                    'image_url' #need to distinguish between teacher and meditation images
                    ]

In [ ]:
#For each batch data file in the teacher_batch_files subdirectory
batch_files_list = os.listdir('../data/med_detail_batch_files')

batch_df_list = []

for batch_file in batch_files_list:
    batch_df = pd.read_csv('../data/med_detail_batch_files/' + batch_file, index_col = 0)    

    #Verify the batch has the correct columns in the correct order.
    correct_columns = True

    batch_columns = list(batch_df.columns)
    if len(batch_columns) == len(expected_columns):
        for col_position in range(0,len(batch_columns)):
            if batch_columns[col_position] != expected_columns[col_position]:
                correct_columns = False
    else:
        correct_columns = False
        
#    batch_df_list = batch_df_list.append(batch_df)
    if correct_columns:
        batch_df_list = batch_df_list + [batch_df]
    else:
        print('ERROR COLUMNS NOT AS EXPECTED',batch_file)

meditations_df = pd.concat(batch_df_list)

In [ ]:
meditations_df.shape

## Identify bad batches

In [ ]:
errors_by_batch = meditations_df.loc[meditations_df.med_detail_scrape_status != 'title found']
errors_by_batch = errors_by_batch[['med_detail_batch_id','med_detail_scrape_status']]
errors_by_batch = errors_by_batch.groupby(by=['med_detail_batch_id']).agg('count')
errors_by_batch = errors_by_batch.rename(columns={'med_detail_scrape_status':'rows_with_med_or_title_not_found'})
errors_by_batch = errors_by_batch.reset_index()
errors_by_batch = errors_by_batch.sort_values(by=['rows_with_med_or_title_not_found'],ascending=False)
errors_by_batch.head(20)

## Cleansing Rows
- Remove duplicates
- Remove page not found
- Remove title not found

### Page not found

In [ ]:
#Show meditations whose page could not be found.
page_not_found_rows= meditations_df.loc[meditations_df.med_detail_scrape_status == 'page not found'].copy()
page_not_found_rows

In [ ]:
#Remove teachers whose page could not be found.
meditations_df = meditations_df.loc[meditations_df.med_detail_scrape_status != 'page not found']
meditations_df.shape

### Title not found

In [ ]:
#Show teachers whose name could not be found. Try rerunning their batches to try to get their info again.
title_not_found_rows = meditations_df.loc[meditations_df.med_detail_scrape_status == 'title not found'].copy()
title_not_found_rows

In [ ]:
#Remove teachers whose name could not be found.
meditations_df = meditations_df.loc[meditations_df.med_detail_scrape_status != 'title not found']
meditations_df.shape

### Duplicates

In [ ]:
meditations_df = meditations_df.sort_values(by='meditation_id')
meditations_df.loc[meditations_df[['meditation_id']].duplicated()]

In [ ]:
meditations_df = meditations_df.loc[~meditations_df[['meditation_id']].duplicated()]
meditations_df.shape

## Cleansing Columns

### Take subset of columns

In [ ]:
meditations_df = meditations_df[['teacher_id',
                                 'meditation_id',
                                #'med_list_batch_id',        Removing this column
                                #'med_list_scrape_date',     Removing this column
                                #'med_list_scrape_status',   Removing this column
                                 'med_detail_batch_id',      
                                 'med_detail_scrape_date',
                                #'med_detail_scrape_status', Removing this column
                                 'title',
                                 'upload_date',
                                 'duration',
                                 'plays',
                                 'rating',
                                 'reviews',
                                 'track_type',
                                 'activity',
                                 'suitable_for',
                                 'topics',
                                 'description',
                                 'meditation_url',
                                 'image_url']]

In [ ]:
for index, row in meditations_df.iterrows():
    
    #teacher_id -- no changes
    
    #meditation_id -- remove prefix
    if row.meditation_id[:20] == '/guided-meditations/':
        meditations_df.loc[index, 'meditation_id'] = row.meditation_id[20:]
    
    #med_detail_scrape_date   -- no changes
    #title                    -- no changes
    #upload_date              -- no changes
    
    #duration -- Example values: PT41M2S, PT6M28S, PT1H3S, PT1H, PT54S, PT11M
    if pd.notna(row.duration):
        med_hours = re.search("\d+H", row.duration)
        if med_hours is None:
            med_hours = int(0)
        else:
            med_hours = int(med_hours.group()[:-1])
        med_minutes = re.search("\d+M", row.duration)
        if med_minutes is None:
            med_minutes = int(0)
        else:
            med_minutes = int(med_minutes.group()[:-1])
        med_seconds = re.search("\d+S", row.duration)
        if med_seconds is None:
            med_seconds = int(0)
        else:
            med_seconds = int(med_seconds.group()[:-1])
        meditations_df.loc[index, 'duration'] = timedelta(hours=med_hours,
                                                          minutes=med_minutes,
                                                          seconds=med_seconds)
    
    #plays -- Example values: 465, 45k, 3m
    if str(row.plays)[-1:] == 'k':
        num_plays = int(float(row.plays[:-1]) * 1000)
    elif str(row.plays)[-1] == 'm':
        num_plays = int(float(row.plays[:-1]) * 1000000)
    elif pd.isna(row.plays):
        num_plays = 0
    else:
        num_plays = int(row.plays)
    meditations_df.loc[index, 'plays'] = int(num_plays)
    
    #rating         -- Example value: 4.63  Handling for null when change type to int?
    #reviews
    if pd.isna(row.reviews):
        meditations_df.loc[index, 'reviews'] = int(0)
    #track_type     -- no changes -- Values: guided, talks, music
    #activity       -- no changes -- value; meditation  (Are there others?)
    #suitable_for   -- no changes -- values: Everyone, Children, Experienced, Beginners
    
#topics -- separate table, will do later in notebook
    
    #description    -- no changes
    #meditation_url -- no changes
    #image_url      -- no changes

In [ ]:
meditations_df.loc[pd.isna(meditations_df.reviews)]

### Set correct data types

In [ ]:
meditations_df['med_detail_scrape_date'] = pd.to_datetime(meditations_df.med_detail_scrape_date)
meditations_df['upload_date'] = pd.to_datetime(meditations_df.upload_date)
meditations_df['duration'] = pd.to_timedelta(meditations_df.duration)

In [ ]:
#Datatypes dictionary
meditation_datatypes = {'teacher_id':'str',
                        'meditation_id':'str',
                        'med_detail_batch_id':'int',
                       #'med_detail_scrape_date'
                        'title':'str',
                        #'upload_date'
                        #'duration''
                        'plays':'int',
                        'rating':'float', 
                        'reviews':'int',
                        'track_type':'str',
                        'activity':'str',
                        'suitable_for':'str',
                        'topics': 'str',
                        'description':'str',
                        'meditation_url':'str',
                        'image_url':'str'}

meditations_df = meditations_df.astype(dtype=meditation_datatypes)

In [ ]:
meditations_df.info()

### Save meditations data to csv

In [ ]:
meditations_df.to_csv('../data/meditations_df.csv')

## Topics

### Create meditations_topics_df

In [ ]:
meditations_df = meditations_df.sort_values(by=['med_detail_batch_id'])
current_batch = 0

meditation_id_list = []
topic_list = []


for index, row in meditations_df.iterrows():
    if row.med_detail_batch_id != current_batch:
        current_batch = row.med_detail_batch_id
        print('Batch ID =',current_batch)
    row_topics = row.topics.split(',')
    
    for topic in row_topics:
        meditation_id_list = meditation_id_list + [row.meditation_id]
        if topic == 'nan':
            topic_list = topic_list + ['Unknown']
        else:
            topic_list = topic_list + [topic]
        
meditations_topics_df = pd.DataFrame({'meditation_id':meditation_id_list, 'topic':topic_list})

In [ ]:
meditations_topics_df.head()

In [ ]:
meditations_topics_df.to_csv('../data/meditations_topics_df.csv')

### Create topics_df

In [ ]:
topics_df = pd.read_csv('../data/topics_df.csv',index_col=0)
topics_df.head()

In [ ]:
missing_topics = meditations_topics_df.merge(topics_df,how='left',on='topic')
missing_topics = missing_topics.loc[(missing_topics.category.isna()) & (missing_topics.subcategory.isna())]
missing_topics.topic.unique()

In [ ]:
topics_df = topics_df.sort_values(by=['category','subcategory','topic'])
topics_df = topics_df.reset_index(drop=True)

In [ ]:
topics_df.to_csv('../data/topics_df.csv')

Some topics are placed in more than one subcategory:
- Stress in Sleep, Stress & Anxiety
- Workplace in Performance, Stress & Anxiety
- Anxiety in Sleep, Stress & Anxiety
- Relax in Health & Happiness, Sleep
- Kids Sleep in For Parents, Sleep